In [ ]:
## LIBRARIES
# ---

# Selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains

# Options driver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

# Dataframes
import pandas as pd
import itertools
import os

# Simulating human behavior
import time
from time import sleep
import random

# Clear data
import unidecode

# Json files
import json
import re
import numpy as np
import itertools
from pandas import json_normalize

# To use explicit waits
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Download files
import urllib.request
import requests
from openpyxl import Workbook

# pytesseract
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [ ]:
def scraper_SUNEDU( df ):
    
    options = Options()
    service = Service( ChromeDriverManager().install( ) )
    driver  = webdriver.Chrome( service = service )
    driver.maximize_window()

    url     = f'https://enlinea.sunedu.gob.pe/'
    driver.get( url )
    wait    = WebDriverWait( driver, 30 )
    
    verificar_titulos_popup = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' ) ) )
    verificar_titulos_popup.click()
    
    frame_popup = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="ifrmShowFormConstancias"]' ) ) )
    driver.switch_to.frame( frame_popup )
    
    variable_flag = False
    
    for index, row in df.iterrows():
        
        dni_valor = row[ 'dni' ]
        print( f'DNI: { dni_valor }' )
        
        dni_campo = WebDriverWait( driver, 60 ).until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="doc"]' ) ) )
        dni_campo.clear()
        dni_campo.send_keys( dni_valor )
        
        max_retries = 15

        for retry in range( max_retries ):
        
            try: 

                time.sleep( 2 )
                captcha       = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="captchaImg"]/img' ) ) )
                captcha_image = captcha.screenshot_as_png
                
                ruta_img      = f'img/img.png'
                with open( ruta_img, 'wb' ) as f:
                    f.write( captcha_image )

                clave_valor   = pytesseract.image_to_string( Image.open( ruta_img ),
                                                             config = '-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789' ).strip()
                campo_captcha = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="captcha"]' ) ) )
                campo_captcha.clear()
                campo_captcha.send_keys( clave_valor )   

                buscar_boton = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="buscar"]' ) ) )
                buscar_boton.click()
                
                time.sleep( 10 )
                
                try: 
                    
                    grado         = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[1]/p' ) ) ).text
                    fecha_diploma = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/b[2]' ) ) ).text
                    modalidad     = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[2]/span' ) ) ).text
                    institucion   = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="finalData"]/tr[1]/td[3]/span' ) ) ).text

                    ruta_img_nuev = os.path.join( os.path.dirname( ruta_img ), f'{ clave_valor }.png' )
                    os.rename( ruta_img, ruta_img_nuev )

                    print( f'Retry: { retry }' )
                    print( f'Clave: { clave_valor }' )
                    print( f'Grado: { grado }' )
                    print( f'Fecha del diploma: { fecha_diploma }' )
                    print( f'Modalidad: { modalidad }' )
                    print( f'Institutción: { institucion }' )
                    print( '\n' )    

                    df.at[ index, 'grado' ]         = grado       
                    df.at[ index, 'fecha_diploma' ] = fecha_diploma    
                    df.at[ index, 'modalidad' ]     = modalidad    
                    df.at[ index, 'institucion' ]   = institucion

                    driver.switch_to.default_content()
                    driver.refresh()
                    
                    verificar_titulos_popup = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' ) ) )
                    verificar_titulos_popup.click()
                    
                    frame_popup = wait.until( EC.presence_of_element_located( ( By.XPATHBy.XPATH, '//*[@id="ifrmShowFormConstancias"]') ) )
                    driver.switch_to.frame(frame_popup)            

                    break                    
                
                except:   
                
                    error_popup        = WebDriverWait (driver, 1 ).until( EC.visibility_of_element_located( ( By.XPATH, '//*[@id="frmError"]' ) ) )
                    class_error_popup  = error_popup.get_attribute( 'class' )
                    caso_no_resultados = 'No se encontraron resultados con este número de DNI'
                    error_msj          = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="frmError_Body"]/p' ) ) ).text
                    
                    if error_msj.startswith( caso_no_resultados ) == False:
                        
                        raise Exception( 'Continuar hacia el bloque except' )
                    
                    else:
                        
                        print( f'No se encontraron resultados' )
                        print( '\n' )   
                        
                        df.at[ index, 'grado' ]         = error_msj       
                        df.at[ index, 'fecha_diploma' ] = error_msj    
                        df.at[ index, 'modalidad' ]     = error_msj    
                        df.at[ index, 'institucion' ]   = error_msj
                        
                        driver.switch_to.default_content()
                        driver.refresh()
                        
                        verificar_titulos_popup = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="dvEnLinea"]/div[2]/div[3]/div/div[2]/div/a' ) ) )
                        verificar_titulos_popup.click()
                        
                        frame_popup = wait.until( EC.presence_of_element_located( ( By.XPATHBy.XPATH, '//*[@id="ifrmShowFormConstancias"]' ) ) )
                        driver.switch_to.frame( frame_popup )    
                        
                        variable_flag = True                        
                        
                        break
                    
                if variable_flag:
                    
                    break
                
            except Exception as e:
                
                # print( f'Error: { e }' )
                print( f'Error en la ejecución' )
                print( f'Intentando de nuevo después de 5 segundos...' )
                print( f'\n' )
                time.sleep( 5 )
                boton_x = wait.until( EC.presence_of_element_located( ( By.XPATH, '//*[@id="closeModalError"]' ) ) )
                boton_x.click()
                
                continue
        
    driver.quit()
        
    return df

In [ ]:
data = pd.read_excel( 'input/data.xlsx', dtype = { 'dni': str } )

In [ ]:
updated_data = scraper_SUNEDU( data )

In [ ]:
updated_data

In [ ]:
updated_data.to_excel( 'output/updated_data.xlsx' )